In [1]:
import tensorflow as tf
from tensorflow import keras

2022-11-01 14:40:56.036840: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-01 14:40:56.101186: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-01 14:40:56.117990: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-01 14:40:56.466846: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: li

In [3]:
!wget http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
!unzip -q spa-eng.zip

/usr/bin/zsh: /home/rayed/anaconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
wget: /home/rayed/anaconda3/envs/tf/lib/libuuid.so.1: no version information available (required by wget)
--2022-11-01 14:41:32--  http://storage.googleapis.com/download.tensorflow.org/data/spa-eng.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 2404:6800:4001:809::2010, 2404:6800:4001:80a::2010, 2404:6800:4001:806::2010, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|2404:6800:4001:809::2010|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2638744 (2.5M) [application/zip]
Saving to: ‘spa-eng.zip’

spa-eng.zip         100%[===================>]   2.52M  3.82MB/s    in 0.7s    

2022-11-01 14:41:33 (3.82 MB/s) - ‘spa-eng.zip’ saved [2638744/2638744]

/usr/bin/zsh: /home/rayed/anaconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)


In [7]:
!shuf -n 10 spa-eng/spa.txt

/usr/bin/zsh: /home/rayed/anaconda3/envs/tf/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Can you meet tonight?	¿Te puedes juntar esta noche?
She gave in to the temptation.	Ella cedió a la tentación.
I don't have a sword.	Yo no tengo una espada.
Look out for bees.	Cuidado con las abejas.
Tom went out to look for something to eat.	Tom salió a buscar algo para comer.
You don't usually do that by yourself, do you?	Normalmente no lo haces tú mismo, ¿verdad?
Tom got here an hour before Mary.	Tom llegó aquí una hora antes que Mary.
If it were not for air, we could not live on the earth.	Si no fuera por el aire, no podríamos vivir en la Tierra.
What did you hope to accomplish?	¿Qué esperabas lograr?
He will also go.	Él también irá.


In [24]:
text_pairs = []
for line in open('spa-eng/spa.txt').readlines():
    eng, spa = line.strip('\n').split('\t')
    spa = '[start] ' + spa + ' [end]'
    text_pairs.append((eng, spa))

In [30]:
import random

random.choice(text_pairs)

('That castle is beautiful.', '[start] Ese castillo es hermoso. [end]')

In [31]:
len(text_pairs)

118964

In [32]:
import random
random.shuffle(text_pairs)
num_val_samples = int(0.15 * len(text_pairs))
num_train_samples = len(text_pairs) - 2 * num_val_samples
train_pairs = text_pairs[:num_train_samples]
val_pairs = text_pairs[num_train_samples:num_train_samples + num_val_samples]
test_pairs = text_pairs[num_train_samples + num_val_samples:]


In [35]:
len(train_pairs), len(val_pairs), len(test_pairs)

(83276, 17844, 17844)

In [41]:
import string
import re

strip_chars = string.punctuation + "¿"
strip_chars = strip_chars.replace('[', '')
strip_chars = strip_chars.replace(']', '')

vocab_size = 15_000
sequence_length = 20
def custom_standardization(input_string):
    lowercase = tf.strings.lower(input_string)
    return tf.strings.regex_replace(lowercase, f'[{re.escape(strip_chars)}]', '')

source_vectorization = keras.layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length
)
target_vectorization = keras.layers.TextVectorization(
    max_tokens = vocab_size,
    output_mode = 'int',
    output_sequence_length = sequence_length + 1,
    standardize = custom_standardization
)

train_english_texts = [pair[0] for pair in train_pairs]
train_spanish_texts = [pair[1] for pair in train_pairs]
source_vectorization.adapt(train_english_texts)
target_vectorization.adapt(train_spanish_texts)

2022-11-01 20:20:49.447038: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 20:20:49.449837: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 20:20:49.449918: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-01 20:20:49.450300: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compi

In [76]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        'english': eng,
        'spanish': spa[:, :-1]
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=8)
    dataset = dataset.shuffle(2048).prefetch(16).cache()
    return dataset

train_ds = make_dataset(train_pairs)
val_ds = make_dataset(val_pairs)

In [78]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


2022-11-01 21:10:14.313725: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [93]:
embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=[None, ], dtype='int64', name='english')
x = keras.layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = keras.layers.Bidirectional(
    keras.layers.GRU(latent_dim), merge_mode='sum'
)(x)

In [94]:
past_target = keras.Input(shape=[None, ], dtype='int64', name='spanish')
x = keras.layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = keras.layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = keras.layers.Dropout(0.5)(x)
target_next_step = keras.layers.Dense(vocab_size, activation='softmax')(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [96]:
input = keras.Input(shape=[4, 5])
dense = keras.layers.Dense(10)
dense(input)

<KerasTensor: shape=(None, 4, 10) dtype=float32 (created by layer 'dense_4')>

In [97]:
seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
seq2seq_rnn.fit(train_ds, epochs=15, validation_data=val_ds)

Epoch 1/15


2022-11-03 11:22:53.925413: W tensorflow/core/common_runtime/forward_type_inference.cc:332] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_41/output/_22'
2022-11-03 11:22:54.376653: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8100
2022-11-03 11:22:54.894324: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


1302/1302 [==============================] - 62s 44ms/step - loss: 1.6424 - accuracy: 0.4156 - val_loss: 1.3155 - val_accuracy: 0.5065
Epoch 2/15
1302/1302 [==============================] - 54s 42ms/step - loss: 1.3198 - accuracy: 0.5253 - val_loss: 1.1582 - val_accuracy: 0.5675
Epoch 3/15
1302/1302 [==============================] - 56s 43ms/step - loss: 1.1765 - accuracy: 0.5758 - val_loss: 1.0738 - val_accuracy: 0.6020
Epoch 4/15
1302/1302 [==============================] - 52s 40ms/step - loss: 1.0836 - accuracy: 0.6086 - val_loss: 1.0338 - val_accuracy: 0.6209
Epoch 5/15
1302/1302 [==============================] - 57s 44ms/step - loss: 1.0345 - accuracy: 0.6335 - val_loss: 1.0212 - val_accuracy: 0.6313
Epoch 6/15
1302/1302 [==============================] - 54s 41ms/step - loss: 1.0050 - accuracy: 0.6512 - val_loss: 1.0179 - val_accuracy: 0.6361
Epoch 7/15
1302/1302 [==============================] - 51s 39ms/step - loss: 0.9847 - accuracy: 0.6649 - val_loss: 1.0163 - val_accura

In [100]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20

def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = '[start]'
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization([decoded_sentence])
        next_token_prediction = seq2seq_rnn.predict([tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(next_token_prediction[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += ' ' + sampled_token
        if sampled_token == '[end]':
            break
    return decoded_sentence

test_eng_texts = [pair[0] for pair in test_pairs]
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print('-')
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
I will miss you.
1/1 [==============================] - 0s 12ms/step
[start] te voy a [UNK] [end]
-
How many people are in this room?
1/1 [==============================] - 0s 14ms/step
[start] cuántos personas hay en esta habitación [end]
-
I think you should do it.
1/1 [==============================] - 0s 12ms/step
[start] creo que deberías hacerlo [end]
-
The royal wedding was a magnificent occasion.
1/1 [==============================] - 0s 12ms/step
[start] la [UNK] de la era una mujer era [UNK] [end]
-
I can't finish this today.
1/1 [==============================] - 0s 12ms/step
[start] no puedo hoy esto [end]
-
I don't think it'll be that easy.
1/1 [==============================] - 0s 12ms/step
[start] no creo que sea fácil [end]
-
This hat suits me nicely.
1/1 [==============================] - 0s 12ms/step
[start] este sombrero me parece que me [UNK] [end]
-
This is very bad.
1/1 [==============================] - 0s 11ms/step
[start] esto es muy mal [end]
-
This is all a

In [153]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
  
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
  
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

In [157]:
class TransformerDecoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention_1 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.attention_2 = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
        self.layernorm_3 = layers.LayerNormalization()
        self.supports_masking = True

    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

    def get_causal_attention_mask(self, inputs):
        input_shape = tf.shape(inputs)
        batch_size, sequence_length = input_shape[0], input_shape[1]
        i = tf.range(sequence_length)[:, tf.newaxis]
        j = tf.range(sequence_length)
        mask = tf.cast(i >= j, dtype="int32")
        mask = tf.reshape(mask, (1, input_shape[1], input_shape[1]))
        mult = tf.concat(
            [tf.expand_dims(batch_size, -1),
             tf.constant([1, 1], dtype=tf.int32)], axis=0)
        return tf.tile(mask, mult)

    def call(self, inputs, encoder_outputs, mask=None):
        causal_mask = self.get_causal_attention_mask(inputs)
        if mask is not None:
            padding_mask = tf.cast(
                mask[:, tf.newaxis, :], dtype="int32")
            padding_mask = tf.minimum(padding_mask, causal_mask)
        attention_output_1 = self.attention_1(
            query=inputs,
            value=inputs,
            key=inputs,
            attention_mask=causal_mask)
        attention_output_1 = self.layernorm_1(inputs + attention_output_1)
        attention_output_2 = self.attention_2(
            query=attention_output_1,
            value=encoder_outputs,
            key=encoder_outputs,
            attention_mask=padding_mask,
        )
        attention_output_2 = self.layernorm_2(
            attention_output_1 + attention_output_2)
        proj_output = self.dense_proj(attention_output_2)
        return self.layernorm_3(attention_output_2 + proj_output)


In [158]:
class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
  
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
 
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
 
    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [159]:
embed_dim = 256 
dense_dim = 2048 
num_heads = 8 
  
encoder_inputs = keras.Input(shape=(None,), dtype="int64", name="english")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(encoder_inputs)
encoder_outputs = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
 
decoder_inputs = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = PositionalEmbedding(sequence_length, vocab_size, embed_dim)(decoder_inputs)
x = TransformerDecoder(embed_dim, dense_dim, num_heads)(x, encoder_outputs)
x = layers.Dropout(0.5)(x)
decoder_outputs = layers.Dense(vocab_size, activation="softmax")(x)
transformer = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [160]:
transformer.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"])
transformer.fit(train_ds, epochs=30, validation_data=val_ds)

Epoch 1/30
1302/1302 [==============================] - 46s 34ms/step - loss: 1.6832 - accuracy: 0.4155 - val_loss: 1.3282 - val_accuracy: 0.5108
Epoch 2/30
1302/1302 [==============================] - 42s 32ms/step - loss: 1.3401 - accuracy: 0.5326 - val_loss: 1.1576 - val_accuracy: 0.5747
Epoch 3/30
1302/1302 [==============================] - 42s 33ms/step - loss: 1.1833 - accuracy: 0.5846 - val_loss: 1.0813 - val_accuracy: 0.6031
Epoch 4/30
1302/1302 [==============================] - 44s 33ms/step - loss: 1.0925 - accuracy: 0.6170 - val_loss: 1.0405 - val_accuracy: 0.6247
Epoch 5/30
1302/1302 [==============================] - 43s 33ms/step - loss: 1.0431 - accuracy: 0.6396 - val_loss: 1.0128 - val_accuracy: 0.6384
Epoch 6/30
1302/1302 [==============================] - 43s 33ms/step - loss: 1.0107 - accuracy: 0.6560 - val_loss: 1.0021 - val_accuracy: 0.6451
Epoch 7/30
1302/1302 [==============================] - 42s 33ms/step - loss: 0.9873 - accuracy: 0.6687 - val_loss: 0.9979 -

In [161]:
import numpy as np
spa_vocab = target_vectorization.get_vocabulary()
spa_index_lookup = dict(zip(range(len(spa_vocab)), spa_vocab))
max_decoded_sentence_length = 20 
  
def decode_sequence(input_sentence):
    tokenized_input_sentence = source_vectorization([input_sentence])
    decoded_sentence = "[start]" 
    for i in range(max_decoded_sentence_length):
        tokenized_target_sentence = target_vectorization(
            [decoded_sentence])[:, :-1]
        predictions = transformer(
            [tokenized_input_sentence, tokenized_target_sentence])
        sampled_token_index = np.argmax(predictions[0, i, :])
        sampled_token = spa_index_lookup[sampled_token_index]
        decoded_sentence += " " + sampled_token
        if sampled_token == "[end]":
            break
    return decoded_sentence
  
test_eng_texts = [pair[0] for pair in test_pairs] 
for _ in range(20):
    input_sentence = random.choice(test_eng_texts)
    print("-")
    print(input_sentence)
    print(decode_sequence(input_sentence))

-
Don't rely too much on your guidebook.
[start] no te [UNK] mucho en tu [UNK] [end]
-
The movie is showing in movie theaters next month.
[start] la película me está pasando en la [UNK] del mes que viene [end]
-
Seicho Matumoto died in 1992.
[start] la [UNK] de la sala de [UNK] [end]
-
That child wants some friends to play with.
[start] ese niño quiere algunos amigos con al juego [end]
-
I thought Tom would show up.
[start] pensé que tom [UNK] [end]
-
My father stopped smoking.
[start] mi padre se ha de fumar [end]
-
This chair is soft and comfortable.
[start] esta silla está se de de de de de de de lo [UNK] [end]
-
Tom'll find out.
[start] tom [UNK] [end]
-
After the game, he went straight home to feed his dog.
[start] después de que él se fue [UNK] dar a su perro de comer [end]
-
Tom has three adult sons.
[start] tomás tiene tres hijos más que hijos [end]
-
Tom wouldn't know.
[start] tomás no se lo sabe [end]
-
Let's play baseball.
[start] vamos a jugar al béisbol [end]
-
Somebody an